# Aufgabe 3: Inverse Filterung
Ist die einem aufgenommenen Bild $g = (f * h) + n$ zugrundeliegende Störfunktion $h$ und deren Fouriertransformierte $H = \mathcal{F}(h)$ bekannt, so kann mittels *inverser Filterung* die Fouriertransformierte des Originalbildes $f$ rekonstruiert werden:
\begin{align}
  \widehat{F}(u, v) &= \frac{G(u,v)}{H(u,v)} \,.
\end{align}
Dabei ist
\begin{align}
  \widehat{F}(u, v) &= F(u,v) + \frac{N(u,v)}{H(u,v)}
\end{align}
weiterhin durch einen unbekannten additiven Rauschprozess $n$ gestört.

Modellieren Sie einen selbstgewählten Störprozess (beispielsweise die durch Bewegung der Kamera während der Bildaufnahme erzeugte Bewegungsunschärfe oder atmosphärische Störungen, siehe Vorlesungsfolien) und simulieren Sie dessen Auswirkungen auf einigen Beispielbildern!

Versuchen Sie nun, mittels inverser Filterung das Originalbild zu rekonstruieren!
Wiederholen Sie diesen Versuch mit verschiedenen den eigentlichen Störprozess begleitenden Rauschprozessen und variieren Sie deren Intensitäten!
Beschreiben und erklären Sie Ihre Beobachtungen!

## 0. Pfade, Pakete etc.

In [ ]:
import glob

%matplotlib inline
import matplotlib.pyplot as plt

import imageio
import numpy as np
import math

In [ ]:
image_filter = 'Bilder/*.jpg'

## 1. Definition der Störprozesse

Definieren Sie den **Störprozess** als Funktion $H$ der Koordinaten $(u,v)$ im Frequenzbereich.  
Gehen Sie dabei von normalisierten Koordinaten $u, v \in [-1,1]$ aus!

In [ ]:
H = lambda u,v: ...

Geben Sie nun einen additiven **Rauschprozess** als Funktion $N$ an, die ebenfalls im Frequenzbereich vorliegt.

*Hinweis: Normalverteiltes Rauschen ist im Frequenzbereich ebenfalls normalverteilt (sowohl im reellen als auch imaginären Bestandteil).*

In [ ]:
N = lambda u, v: np.complex(..., ...)

## 2. Laden des Bildes

In [ ]:
image_path = np.random.choice(glob.glob(image_filter))
image = imageio.imread(image_path)

Für diese Aufgabe ist es wichtig, das Bild im Fließkommaformat vorliegen zu haben. Andernfalls kann nicht immer korrekt gerechnet werden. Konvertieren sie `image` zu einer geeigneten Repräsentation:

## 3. Simulation der Störung

Wir definieren zunächst eine Hilfsfunktion `ex4_apply_noise`, die ein Originalbild $f$ mit den Prozessen $H$ und $N$ stört und das gestörte Bild $g$ zurückgibt:

In [ ]:
def ex4_apply_noise(f, H, N):
    # Fouriertransformation
    F = np.fft.fftshift(np.fft.fft2(f))
    
    G = np.zeros_like(F)
    
    for u, v in np.ndindex(*f.shape):
        u_ = 2 * (float(u) / f.shape[0]) - 1.0
        v_ = 2 * (float(v) / f.shape[1]) - 1.0
        
        # Anwendung des Prozesses N und H
        G[u,v] = F[u,v] * H(u_, v_) + N(u_, v_)
    
    g = np.real(np.fft.ifft2(np.fft.ifftshift(G)))
    return g

Die Funktion wird nun eingesetzt, um das Bild einmal nur durch $H$ und einmal durch $H$ und $N$ zu stören.

In [ ]:
g = ex4_apply_noise(image, H, lambda u, v: 0)  # Störung nur durch H, N ist 0
g_noise = ex4_apply_noise(image, H, N)

Visualisieren Sie `image`, `g` und `g_noise` nebeneinander:

## 4. Rekonstruktion
Es soll nun versucht werden, das Originalbild $f$ aus `g` und `g_noise` mittels inverser Filterung wiederherzustellen. Dazu definieren wir zunächst eine Funktion `ex4_inverse_filter`, die mit Hilfe der Gleichung (1) eine solche durchführt und das entstörte Bild $\hat{f}$ zurückgibt. Dabei ist diesmal allerdings nur $H$ bekannt:

In [ ]:
def ex4_inverse_filter(g, H):
    ...
    return f_

Die Funktion wird jetzt für `g1` und `g2` aufgerufen:

In [ ]:
recons = ex4_inverse_filter(g, H)
recons_noise = ex4_inverse_filter(g_noise, H)

Visualisieren Sie nun das Originalbild `image` neben den Rekonstruktionen `recons` und `recons_noise`: